# No Show EDA

Exploratory data analysis of first datadump of no-show

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

from noshow.preprocessing.load_data import process_appointments, process_postal_codes, load_appointment_csv
from noshow.visualisation.features_plots import feature_barplot, feature_scatter
from noshow.features.no_show_features import prev_no_show_features
from noshow.features.appointment_features import add_days_since_created, add_appointments_same_day, add_minutes_early, add_time_features, add_days_since_last_appointment, add_appointments_last_days
from noshow.features.patient_features import add_patient_features

## Read and clean data

In [ ]:
appointments_df = load_appointment_csv("../data/raw/poliafspraken_no_show.csv")
appointments_df = process_appointments(appointments_df)
appointments_df.shape


## Plot no-shows

In [ ]:
cancelation_reason_count = (
    appointments_df[appointments_df["no_show"] == "no_show"]
    .groupby("cancelationReason_display")["no_show"]
    .count()
)

fig, ax = plt.subplots(figsize=(10, 8))

bar_container = ax.bar(cancelation_reason_count.index, cancelation_reason_count)
ax.bar_label(bar_container)
ax.set_xticks(range(len(cancelation_reason_count.index)))
ax.set_xticklabels(cancelation_reason_count.index, rotation=45, ha="right")
ax.set_title("Cancelation reasons for no-shows")
fig.set_tight_layout("h_pad")
plt.show()

In [ ]:
no_show_per_specialisation = (
    appointments_df.groupby("specialty_code")["no_show"].value_counts().unstack()
)

bottom = np.zeros(2)
fig, ax = plt.subplots(figsize=(10, 4))
for idx, row in no_show_per_specialisation.iterrows():
    ax.bar(["no-show", "show"], row, label=idx, bottom=bottom)
    bottom += row
ax.legend()

plt.show()


In [ ]:
no_show_percent = (
    appointments_df.groupby("no_show")["specialty_code"]
    .value_counts(normalize=True)
    .unstack(level="no_show")
)
fig, ax = plt.subplots(figsize=(10, 4))

bottom = np.zeros(2)
for idx, row in no_show_percent.iterrows():
    ax.bar(["no-show", "show"], row, label=idx, bottom=bottom)
    bottom += row

ax.legend()

plt.show()


In [ ]:
no_show_over_time = appointments_df.copy().reset_index()
no_show_over_time["start"] = (
    no_show_over_time["start"].dt.to_period("Y").dt.to_timestamp()
)
no_show_over_time = (
    no_show_over_time.groupby(["specialty_code", "start"])["no_show"]
    .value_counts(normalize=False)
    .unstack(level="no_show")
)

fig, ax = plt.subplots(figsize=(12, 5))

for idx in no_show_over_time.index.unique(level="specialty_code"):
    ax.plot(
        no_show_over_time.loc[idx].index,
        no_show_over_time.loc[idx, "no_show"],
        label=idx,
    )
ax.legend()
ax.spines.top.set_visible(False)
ax.spines.right.set_visible(False)
ax.set_title("Number of no-shows per specialty per year")

plt.show()


## Calculate earlier no-shows

In [ ]:
appointments_features = prev_no_show_features(appointments_df)

In [ ]:
feature_barplot(appointments_features, "prev_no_show", feature_name="previous no-shows")
plt.show()

In [ ]:
feature_barplot(
    appointments_features,
    "prev_no_show_perc",
    feature_name="previous no-show percentage",
    perc_feature=True,
    round_decimals=1,
)
plt.show()

In [ ]:
feature_scatter(appointments_features, "earlier_appointments")
plt.show()

### Calculate days since created

In [ ]:
appointments_features = add_days_since_created(appointments_features)


In [ ]:
feature_scatter(appointments_features, "days_since_created")
plt.show()


### Calculate appointments on the same day

In [ ]:
appointments_features = add_appointments_same_day(appointments_features)


In [ ]:
feature_barplot(appointments_features, "appointments_same_day")
plt.show()


In [ ]:
appointments_features = add_days_since_last_appointment(appointments_features)
appointments_features = add_appointments_last_days(appointments_features)


In [ ]:
feature_scatter(appointments_features, "days_since_last_appointment")
plt.show()


In [ ]:
feature_barplot(appointments_features, "appointments_last_days")
plt.show()


## Calculate patient features

### Postal codes

Using the dump from https://download.geonames.org/export/dump/

In [ ]:
all_postalcodes = process_postal_codes("../data/raw/NL.txt")
appointments_features = add_patient_features(appointments_features, all_postalcodes)

### Plot patient features

In [ ]:
feature_scatter(appointments_features, "age")
plt.show()


In [ ]:
feature_scatter(
    appointments_features,
    "dist_umcu",
    feature_name="distance to UMCU",
    round_feature=True,
)
plt.show()

## Calculate punctuality patients

In [ ]:
appointments_features = add_minutes_early(appointments_features)


In [ ]:
appointments_features["minutes_early"].plot.hist(bins=10)


### Plot punctuality vs no show

In [ ]:
feature_scatter(
    appointments_features,
    "prev_minutes_early",
    feature_name="previous minutes too early",
    round_feature=True,
)
plt.show()

## Add time features

In [ ]:
appointments_features = add_time_features(appointments_features)


In [ ]:
feature_barplot(appointments_features, "hour")
plt.show()


In [ ]:
feature_barplot(appointments_features, "weekday")
plt.show()


## Plot categorical features

In [ ]:
ax = feature_barplot(appointments_features, "specialty_code")
ax.tick_params("x", labelrotation=45)
plt.show()


In [ ]:
appointments_df["specialty_code"].value_counts()

## Save featuretable

In [ ]:
appointments_features[
    [
        "hour",
        "weekday",
        "specialty_code",
        "minutesDuration",
        "no_show",
        "prev_no_show",
        "prev_no_show_perc",
        "age",
        "dist_umcu",
        "prev_minutes_early",
        "earlier_appointments",
        "appointments_same_day",
        "days_since_created",
    ]
].to_parquet(
    "/mapr/no_show/no_show_onderzoeker/rpeters7/No_Show/data/processed/featuretable.parquet"
)
